In [ ]:
!pip install transformers
!pip install datasets

     |████████████████████████████████| 3.5 MB 5.3 MB/s 
     |████████████████████████████████| 596 kB 43.5 MB/s 
     |████████████████████████████████| 67 kB 5.3 MB/s 
     |████████████████████████████████| 6.8 MB 34.9 MB/s 
     |████████████████████████████████| 895 kB 44.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 311 kB 5.4 MB/s 
     |████████████████████████████████| 133 kB 47.1 MB/s 
     |████████████████████████████████| 243 kB 44.1 MB/s 
     |████████████████████████████████| 1.1 MB 42.0 MB/s 
     |████████████████████████████████| 271 kB 46.0 MB/s 
     |████████████████████████████████| 144 kB 45.7 MB/s 
     |████████████████████████████████| 94 kB 3.0 MB/s 


In [ ]:
# This data loaded can be our corpus 
from datasets import load_dataset

dataset = load_dataset("xquad","xquad.tr")

Downloading:   0%|          | 0.00/1.96k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.29k [00:00<?, ?B/s]

  0%|          | 0/12 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/168k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/138k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/137k [00:00<?, ?B/s]

  0%|          | 0/12 [00:00<?, ?it/s]

0 examples [00:00, ? examples/s]

Dataset xquad downloaded and prepared to /root/.cache/huggingface/datasets/xquad/xquad.tr/1.0.0/39e1ff0497cbbfb79bbff61024031c10872bbd7c4fd8bc250207a965c39d3336. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
dataset

DatasetDict({
    validation: Dataset({
        features: ['id', 'context', 'question', 'answers'],
        num_rows: 1190
    })
})

In [ ]:
raw_validation_dataset = dataset["validation"]
raw_validation_dataset

Dataset({
    features: ['id', 'context', 'question', 'answers'],
    num_rows: 1190
})

In [ ]:
raw_validation_dataset["answers"][40]

{'answer_start': [15], 'text': ['iki']}

In [ ]:
len(raw_validation_dataset["context"][40])

363

In [ ]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
model_checkpoint = "savasy/bert-base-turkish-squad"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/152 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/485 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/245k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/422M [00:00<?, ?B/s]

In [ ]:
# Preprocessing the validation data
max_length = 384
stride = 128 # Can be tweaked according to the data we use


def preprocess_validation_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

In [ ]:
small_eval_set = raw_validation_dataset.select(range(400))
small_eval_set

Dataset({
    features: ['id', 'context', 'question', 'answers'],
    num_rows: 400
})

In [ ]:
# Applying the preprocess function to the validation dataset
eval_set = small_eval_set.map(
    preprocess_validation_examples,
    batched = True,
    remove_columns = raw_validation_dataset.column_names,
)
len(eval_set)

  0%|          | 0/1 [00:00<?, ?ba/s]

400

In [ ]:
#torch.cuda.empty_cache()
eval_set

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping', 'example_id'],
    num_rows: 400
})

In [ ]:
#torch.cuda.memory_summary(device=None, abbreviated=False)

In [ ]:
# Evaluation
import torch
from transformers import AutoModelForQuestionAnswering

eval_set_for_model = eval_set.remove_columns(["example_id","offset_mapping"])
eval_set_for_model.set_format("torch")

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
batch = {k: eval_set_for_model[k].to(device) for k in eval_set_for_model.column_names}
trained_model =  AutoModelForQuestionAnswering.from_pretrained(model_checkpoint).to(device)

with torch.no_grad():
  outputs = trained_model(**batch)

In [ ]:
#outputs

QuestionAnsweringModelOutput([('start_logits',
                               tensor([[ -2.1165, -11.5495, -11.8514,  ..., -11.6134, -11.5446, -11.5356],
                                       [ -5.5803, -11.5969, -11.6385,  ..., -11.5207, -11.4332, -11.4415],
                                       [ -5.4171, -11.6113, -11.7961,  ..., -11.5422, -11.4944, -11.4594],
                                       ...,
                                       [-11.2503, -11.3530, -11.4619,  ..., -11.5543, -11.5393, -11.5359],
                                       [-10.5655, -11.5574, -11.5506,  ..., -11.5534, -11.5491, -11.5441],
                                       [-11.2890, -11.5323, -11.6290,  ..., -11.5348, -11.5171, -11.5182]],
                                      device='cuda:0')),
                              ('end_logits',
                               tensor([[ -3.1678, -12.4336, -12.1774,  ..., -12.3433, -12.3895, -12.3931],
                                       [ -6.1181, -12.684

In [ ]:
start_logits = outputs.start_logits.cpu().numpy()
end_logits = outputs.end_logits.cpu().numpy()

In [ ]:
eval_set
#small_eval_set

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping', 'example_id'],
    num_rows: 400
})

In [ ]:
import collections

example_to_features = collections.defaultdict(list)
for idx, feature in enumerate(eval_set):
    example_to_features[feature["example_id"]].append(idx)

import numpy as np

n_best = 20
max_answer_length = 30
predicted_answers = []

for example in small_eval_set:
    example_id = example["id"]
    context = example["context"]
    answers = []

    for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = eval_set["offset_mapping"][feature_index]
            #Used for debugging
            #print(feature_index)

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length.
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answers.append(
                        {
                            "text": context[offsets[start_index][0] : offsets[end_index][1]],
                            "logit_score": start_logit[start_index] + end_logit[end_index],
                        }
                    )
    # I added this part                
    if len(answers) == 0:
        best_answer = {"logit_score": -1000,"text": []}
        predicted_answers.append({"id": example_id, "prediction_text": best_answer["text"]})
    else:
        best_answer = max(answers, key=lambda x: x["logit_score"])
        predicted_answers.append({"id": example_id, "prediction_text": best_answer["text"]})

In [ ]:
from datasets import load_metric 

metric = load_metric("squad")

Downloading:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

In [ ]:
theoretical_answers = [
                       {"id": ex["id"], "answers": ex["answers"]} for ex in small_eval_set
]

In [ ]:
metric.compute(predictions=predicted_answers, references=theoretical_answers)

{'exact_match': 47.0, 'f1': 64.5021095417307}

In [ ]:
print(raw_validation_dataset["question"][0])
print(raw_validation_dataset["context"][0])

Panthers savunması kaç sayı bırakmıştır?
﻿Panthers savunması ligdeki derecesi altıncı sırada olarak sadece 308 sayı bıraktı, aynı zamanda NFL'de 24 topu kapma ile ligin başını çekmesi ve dört Pro Bowl seçmeleri ile övünç duydu. Pro Bowl orta çizgi savunmacısı Kawann Short, takımı 11'le birlikte sack ederken, aynı zamanda üç oyuncuyu düşürme ve iki yeniden savunmaya zorladı. Yan hakem Mario Addison, 6½ sack ekledi. Panthers hattında ayrıca, sadece 9 başlangıçta 5 sack eden uç çizgi savunmacısı Kony Ealy ile birlikte 136 kez ile NFL'nin aktif kariyer sack lideri ve 5 kez profesyonel bir top atıcısı olan Jared Allen öne çıkmaktadır. Arkalarında, Panthers'lerin üç adet ikinci hat savunma oyuncusundan ikisi de Pro Bowl'da oynamak için seçildi: Thomas Davis ve Luke Kuechly. Davis, dördünde mecburi oyuncuyu düşürme ve dört topu kapma olmak üzere 5½ sack etmeyi bir araya getirirken, Kuechly top çalmalarda (118) iki mecburi oyuncuyu düşürme ve kendi başına dört pasını kesme ile takıma öncülük e

In [ ]:
predicted_answers

[{'id': '56beb4343aeaaa14008c925b', 'prediction_text': '308'},
 {'id': '56beb4343aeaaa14008c925c', 'prediction_text': '136'},
 {'id': '56beb4343aeaaa14008c925d', 'prediction_text': '118'},
 {'id': '56beb4343aeaaa14008c925e', 'prediction_text': '88'},
 {'id': '56beb4343aeaaa14008c925f',
  'prediction_text': 'Thomas Davis ve Luke Kuechly'},
 {'id': '56d6f3500d65d21400198290', 'prediction_text': '24'},
 {'id': '56d6f3500d65d21400198291',
  'prediction_text': 'Thomas Davis ve Luke Kuechly'},
 {'id': '56d6f3500d65d21400198292', 'prediction_text': 'dört'},
 {'id': '56d6f3500d65d21400198293', 'prediction_text': 'dört'},
 {'id': '56d6f3500d65d21400198294',
  'prediction_text': 'Thomas Davis ve Luke Kuechly'},
 {'id': '56d9992fdc89441400fdb59c', 'prediction_text': 'dört'},
 {'id': '56d9992fdc89441400fdb59e', 'prediction_text': 'Kony Ealy'},
 {'id': '56d9992fdc89441400fdb59f', 'prediction_text': 'Kony Ealy'},
 {'id': '56d9992fdc89441400fdb5a0', 'prediction_text': 'dört'},
 {'id': '56beb7953aeaaa

In [ ]:
example=small_eval_set[0]
ex_id = small_eval_set[0]["id"]
ex_id

context = example["context"]
context
start_logit = start_logits[0]
end_logit = end_logits[0]
offsets = eval_set["offset_mapping"][0]



In [ ]:
small_eval_set[0]

{'answers': {'answer_start': [66], 'text': ['308']},
 'context': "\ufeffPanthers savunması ligdeki derecesi altıncı sırada olarak sadece 308 sayı bıraktı, aynı zamanda NFL'de 24 topu kapma ile ligin başını çekmesi ve dört Pro Bowl seçmeleri ile övünç duydu. Pro Bowl orta çizgi savunmacısı Kawann Short, takımı 11'le birlikte sack ederken, aynı zamanda üç oyuncuyu düşürme ve iki yeniden savunmaya zorladı. Yan hakem Mario Addison, 6½ sack ekledi. Panthers hattında ayrıca, sadece 9 başlangıçta 5 sack eden uç çizgi savunmacısı Kony Ealy ile birlikte 136 kez ile NFL'nin aktif kariyer sack lideri ve 5 kez profesyonel bir top atıcısı olan Jared Allen öne çıkmaktadır. Arkalarında, Panthers'lerin üç adet ikinci hat savunma oyuncusundan ikisi de Pro Bowl'da oynamak için seçildi: Thomas Davis ve Luke Kuechly. Davis, dördünde mecburi oyuncuyu düşürme ve dört topu kapma olmak üzere 5½ sack etmeyi bir araya getirirken, Kuechly top çalmalarda (118) iki mecburi oyuncuyu düşürme ve kendi başına dört pas

In [ ]:
start_indexes = np.argsort(start_logit)[-1: -n_best -1 : -1].tolist()
end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
lol = zip(start_indexes,end_indexes)
list(lol)

[(22, 23),
 (15, 24),
 (21, 25),
 (23, 26),
 (11, 22),
 (0, 21),
 (20, 0),
 (34, 34),
 (24, 55),
 (18, 59),
 (29, 31),
 (71, 20),
 (25, 33),
 (175, 27),
 (100, 71),
 (44, 15),
 (26, 178),
 (115, 12),
 (19, 100),
 (14, 180)]

In [ ]:
for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length.
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answers.append(
                        {
                            "text": context[offsets[start_index][0] : offsets[end_index][1]],
                            "logit_score": start_logit[start_index] + end_logit[end_index],
                        }
                    )

In [ ]:
answers

[{'logit_score': 18.626984, 'text': '1985'},
 {'logit_score': 10.42943, 'text': '1985 yılında'},
 {'logit_score': -0.21295929,
  'text': '1985 yılında bir Antlaşma imzalayarak özel statü kazandı.'},
 {'logit_score': -0.6876154,
  'text': '1985 yılında bir Antlaşma imzalayarak özel statü kazandı'},
 {'logit_score': 5.6739745, 'text': '1985'},
 {'logit_score': -0.9064009, 'text': "1985'te"},
 {'logit_score': -5.662901, 'text': "1985'te İspanya ve Portekiz'den 1994"},
 {'logit_score': -5.9543695, 'text': "1985'te İspanya"},
 {'logit_score': -8.045051, 'text': '2007'},
 {'logit_score': -10.505079,
  'text': "2007 Lizbon Antlaşması'yla AB kurumlarında üye devletlerin nisbi güçlerine küçük değişiklikler yapılması için büyük değişiklik antlaşmaları imzalandı. Kuruluşundan beri, 1972"},
 {'logit_score': -12.074784,
  'text': "2007 Lizbon Antlaşması'yla AB kurumlarında üye devletlerin nisbi güçlerine küçük değişiklikler yapılması için büyük değişiklik antlaşmaları imzalandı. Kuruluşundan beri, 

In [ ]:
best_answer = max(answers, key=lambda x: x["logit_score"])
predicted_answers.append({"id": example_id, "prediction_text": best_answer["text"]})

In [ ]:
best_answer


{'logit_score': 25.996775, 'text': '308'}

In [ ]:
test_text = "Bu cümleyi nasıl tokenize edeceğini merak ediyorum"
tokenized_test_text = tokenizer.tokenize(test_text)
tokenized_test_text

['Bu',
 'cümle',
 '##yi',
 'nasıl',
 'tok',
 '##enize',
 'edeceğini',
 'merak',
 'ediyorum']

In [ ]:
# Using the model
from transformers import pipeline
question_answerer = pipeline("question-answering", model = model_checkpoint)

test_text = "NLP yani Doğal Dil İşleme, doğal dillerin kurallı yapısının çözümlenerek anlaşılması veya yeniden üretilmesi amacını taşır.Bu çözümlemenin insana getireceği kolaylıklar, yazılı dokümanların otomatik çevrilmesi, soru-cevap makineleri, otomatik konuşma ve komut anlama, konuşma sentezi, konuşma üretme, otomatik metin özetleme, bilgi sağlama gibi birçok başlıkla özetlenebilir. Bilgisayar teknolojisinin yaygın kullanımı, bu başlıklardan üretilen uzman yazılımların gündelik hayatımızın her alanına girmesini sağlamıştır. Örneğin, tüm kelime işlem yazılımları birer imlâ düzeltme aracı taşır. Bu araçlar aslında yazılan metni çözümleyerek dil kurallarını denetleyen doğal dil işleme yazılımlarıdır. Batı dillerinde SAPI (Microsoft şirketinin konuşma sentezleyici üretmek amacı ile satışa sunduğu geliştirici program) tabanlı Konuşma sentezleyici bileşenleri, yazılımcıların multimedia (çoklu ortam) sunuları hazırlamaları için hizmete sunulmuştur.Konuşma ve komut anlama yazılımları ise gelecekte insan ve bilgisayar arasındaki klavye, fare gibi veri girişi aygıtlarını ortadan kaldıracak yazılımlardır. Bu gelişmeler makine-insan iletişiminde yeni ve devrimci değişimlere yol açacak ve bilgisayarların daha çok insan tarafından kabul görmesine yol açacaktır."

In [ ]:
test_question = "Doğal dil işleme yazılımları ne işe yarar?"
question_answerer(question = test_question, context = test_text)

{'answer': 'yazılan metni çözümleyerek dil kurallarını denetleyen',
 'end': 663,
 'score': 0.6905831694602966,
 'start': 610}